### Import libraries

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.0+cu113


Source: https://pytorch-geometric.readthedocs.io/en/latest/notes/colabs.html

### Graph Classification with Graph Neural Networks

Graph classification refers to the problem of classifiying entire graphs (in contrast to nodes), given a **dataset of graphs**, based on some structural graph properties. Here, we want to embed entire graphs, and we want to embed those graphs in such a way so that they are linearly separable given a task at hand.

Our task for graph classification is **heart arrhythmia prediction**, in which sensors are represented as nodes and ECG records are represented as graphs, and the task is to infer whether a an ECG record has some kind of heart arrhythmia. 

Dataset:
- PhysioNet 2020 (P20-ECG)
- 6877 patients
- 80K MTS samples
- 12-lead ECG
- Each TS has 2500 timestamps
- 9 classes of heart arrhythmia


Graph Structure:
- Each patient's ECG is a graph 
- 12 leads means 12 nodes per graph
- 6877 patients means 6877 ECG graphs for each patient
- 80K MTS samples **( What does this mean?)**
- Each TS has 2500 time steps mean for each node we will have two properties (a) node features for the patient when the ECG is recorded (b) time series data which reflect the heart's electrical signal over 2500 time steps. **(Are the node features just as blood pressure dynamically recorded at the same time resolution, no right? These are only measured once and stay static for the 2500 time steps?)**
- We are going with fully connected graph with self loops which means each node is connected to every other node in the graph. So the node degree for each node is 11+1 (where 11 are edges to other nodes and 1 is the self loop)
- 9 classes of heart issues exist **(It would be nice if someone could check if the classes are balanced or not, just good to know before modelling)**




### Analysing Network Characteristics of the Dummy Dataset 

In [ ]:
# DUMMY DATASET - We will replace it with our own 
# The TU Dortmund University has collected a wide range of different graph classification datasets, known as the [**TUDatasets**](https://chrsmrrs.github.io/datasets/), which are also accessible via [`torch_geometric.datasets.TUDataset`](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.TUDataset) in PyTorch Geometric.
# Let's load and inspect one of the smaller ones, the **MUTAG dataset**:

import torch
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='data/TUDataset', name='MUTAG')

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Extracting data/TUDataset/MUTAG/MUTAG.zip
Processing...



Dataset: MUTAG(188):
Number of graphs: 188
Number of features: 7
Number of classes: 2

Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[1])
Number of nodes: 17
Number of edges: 38
Average node degree: 2.24
Has isolated nodes: False
Has self-loops: False
Is undirected: True


Done!


Note: We will have to change the y in the Data object accordingly **because we have 9 classes and this dataset has 2 classes** - so this is a binary graph classification task. 

### Splitting the graphs into training and test datasets

In [ ]:
# This dataset provides **188 different graphs**, and the task is to classify each graph into **one out of two classes**.

# By inspecting the first graph object of the dataset, we can see that it comes with **17 nodes (with 7-dimensional feature vectors)** and **38 edges** (leading to an average node degree of 2.24).
# It also comes with exactly **one graph label** (`y=[1]`), and, in addition to previous datasets, provides addtional **4-dimensional edge features** (`edge_attr=[38, 4]`).
# However, for the sake of simplicity, we will not make use of those.

# PyTorch Geometric provides some useful utilities for working with graph datasets, *e.g.*, we can shuffle the dataset and use the first 150 graphs as training graphs, while using the remaining ones for testing:

torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:150]
test_dataset = dataset[150:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 150
Number of test graphs: 38


### Mini Batching of Graphs 

In [ ]:
# Mini batching of graphs
# Adjacency matrices are stacked in a diagonal fashion (creating a giant graph that holds multiple isolated subgraphs),
# and node and target features are simply concatenated in the node dimension:
# PyTorch Geometric automatically takes care of batching multiple graphs into a single giant graph with the help of the torch_geometric.data.DataLoader class: 

from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2636], x=[1188, 7], edge_attr=[2636, 4], y=[64], batch=[1188], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2506], x=[1139, 7], edge_attr=[2506, 4], y=[64], batch=[1139], ptr=[65])

Step 3:
Number of graphs in the current batch: 22
DataBatch(edge_index=[2, 852], x=[387, 7], edge_attr=[852, 4], y=[22], batch=[387], ptr=[23])



### GNN Model definitions for Graph Classification 

In [ ]:
# Training Graph Neural Networks for Graph Classification
# Embed each node by performing multiple rounds of message passing
# Aggregate node embeddings into a unified graph embedding (readout layer)
# Train a final classifier on the graph embedding
 
# There exists multiple readout layers in literature, but the most common one is to simply take the average of node embeddings

# For ex: In GCNConv we use the  ReLU(𝑥)=max(𝑥,0)  activation for obtaining localized node embeddings,
# before we apply our final classifier on top of a graph readout layer.

# PyTorch Geometric provides this functionality via torch_geometric.nn.global_mean_pool, 
# which takes in the node embeddings of all nodes in the mini-batch and the assignment vector batch 
# to compute a graph embedding of size [batch_size, hidden_channels] for each graph in the batch.

# The final architecture for applying GNNs to the task of graph classification then looks as follows and allows for complete end-to-end training:

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import GraphConv


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x


class GAT(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GAT, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x
       
# Define the models
model1 = GCN(hidden_channels=64)
model2= GAT(hidden_channels=64)
model3 = GNN(hidden_channels=64)

# Print them 
print(model1)
print(model2)
print(model3)

GCN(
  (conv1): GCNConv(7, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)
GAT(
  (conv1): GATConv(7, 64, heads=1)
  (conv2): GATConv(64, 64, heads=1)
  (conv3): GATConv(64, 64, heads=1)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)
GNN(
  (conv1): GraphConv(7, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [ ]:
# Set model paramters and model type
def set_model_parameters(model_type, lr=0.01):
    model = model_type
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()
    return model, optimizer, criterion

# Train the model
def train(model, optimizer,criterion):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

# Test the model 
def test(loader, model):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.

# Training and Testing Pipeline 
def running_epochs(model,optimizer,criterion):
    for epoch in range(1, 171):
        train(model,optimizer,criterion )
        train_acc = test(train_loader, model)
        test_acc = test(test_loader, model)
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

        

In [ ]:
# Experiment 1: GCN Baseline 
model, optimizer, criterion = set_model_parameters(model1, lr=0.01)
running_epochs(model,optimizer,criterion)


Epoch: 001, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 002, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 003, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 004, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 005, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 006, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 007, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 008, Train Acc: 0.7200, Test Acc: 0.7632
Epoch: 009, Train Acc: 0.7333, Test Acc: 0.7632
Epoch: 010, Train Acc: 0.7467, Test Acc: 0.7632
Epoch: 011, Train Acc: 0.7133, Test Acc: 0.7895
Epoch: 012, Train Acc: 0.7200, Test Acc: 0.8158
Epoch: 013, Train Acc: 0.7333, Test Acc: 0.7632
Epoch: 014, Train Acc: 0.7267, Test Acc: 0.7895
Epoch: 015, Train Acc: 0.7067, Test Acc: 0.8158
Epoch: 016, Train Acc: 0.7333, Test Acc: 0.7895
Epoch: 017, Train Acc: 0.7267, Test Acc: 0.7895
Epoch: 018, Train Acc: 0.7267, Test Acc: 0.8158
Epoch: 019, Train Acc: 0.7467, Test Acc: 0.7632
Epoch: 020, Train Acc: 0.7533, Test Acc: 0.7632
Epoch: 021, Train Acc: 0.7200, Test Acc:

In [ ]:
# Experiment 2: GAT Baseline 
model, optimizer, criterion = set_model_parameters(model2, lr=0.01)
running_epochs(model,optimizer,criterion)


Epoch: 001, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 002, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 003, Train Acc: 0.6800, Test Acc: 0.7368
Epoch: 004, Train Acc: 0.7600, Test Acc: 0.7632
Epoch: 005, Train Acc: 0.7067, Test Acc: 0.7895
Epoch: 006, Train Acc: 0.7467, Test Acc: 0.7895
Epoch: 007, Train Acc: 0.7067, Test Acc: 0.7895
Epoch: 008, Train Acc: 0.7600, Test Acc: 0.7895
Epoch: 009, Train Acc: 0.7533, Test Acc: 0.7895
Epoch: 010, Train Acc: 0.7533, Test Acc: 0.8421
Epoch: 011, Train Acc: 0.7600, Test Acc: 0.8158
Epoch: 012, Train Acc: 0.7600, Test Acc: 0.7895
Epoch: 013, Train Acc: 0.7467, Test Acc: 0.8158
Epoch: 014, Train Acc: 0.7400, Test Acc: 0.8158
Epoch: 015, Train Acc: 0.7600, Test Acc: 0.7632
Epoch: 016, Train Acc: 0.7600, Test Acc: 0.7632
Epoch: 017, Train Acc: 0.7533, Test Acc: 0.7895
Epoch: 018, Train Acc: 0.7333, Test Acc: 0.8158
Epoch: 019, Train Acc: 0.7600, Test Acc: 0.8421
Epoch: 020, Train Acc: 0.7667, Test Acc: 0.7895
Epoch: 021, Train Acc: 0.7600, Test Acc:

In [ ]:
# Experiment 3: GraphConv Baseline - suggested online in docs 
model, optimizer, criterion = set_model_parameters(model3, lr=0.01)
running_epochs(model,optimizer,criterion)

Epoch: 001, Train Acc: 0.3533, Test Acc: 0.2632
Epoch: 002, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 003, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 004, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 005, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 006, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 007, Train Acc: 0.6533, Test Acc: 0.7368
Epoch: 008, Train Acc: 0.7133, Test Acc: 0.7895
Epoch: 009, Train Acc: 0.7333, Test Acc: 0.8158
Epoch: 010, Train Acc: 0.7733, Test Acc: 0.8684
Epoch: 011, Train Acc: 0.7867, Test Acc: 0.8421
Epoch: 012, Train Acc: 0.7667, Test Acc: 0.8421
Epoch: 013, Train Acc: 0.7867, Test Acc: 0.8421
Epoch: 014, Train Acc: 0.8000, Test Acc: 0.8684
Epoch: 015, Train Acc: 0.7933, Test Acc: 0.8421
Epoch: 016, Train Acc: 0.8067, Test Acc: 0.8684
Epoch: 017, Train Acc: 0.7933, Test Acc: 0.8421
Epoch: 018, Train Acc: 0.7600, Test Acc: 0.7895
Epoch: 019, Train Acc: 0.8133, Test Acc: 0.8421
Epoch: 020, Train Acc: 0.7733, Test Acc: 0.7895
Epoch: 021, Train Acc: 0.8000, Test Acc:

Things to do :
- Plot the training and loss curves 
- Test on the real data for 100 or 200 graphs 
- Compare the resulsts for these 3 GNN only experiments 
- Since there are 9 classes of heart issues that exist- it would be nice if someone could check if the classes are balanced or not, just good to know before modelling


Questions to ask:
- Are the node features just as blood pressure dynamically recorded at the same time resolution, no right? These are only measured once and stay static for the 2500 time steps?
- 80K MTS samples ( What does MTS mean? Multivariate Time Series, seems like it?) I am confused because if there are about 6000 patients (6000 graphs) with 12 leads ( so each graph has 12 nodes) and each patient has 2500 time steps of heart activity recorded ( so at each node there is a 2500 time step of electrical activity?). But where does 80K MTS fit into the picture?  